In [ ]:
###########import packages##########
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn.linear_model import BayesianRidge
from sklearn.tree import ExtraTreeRegressor
from sklearn.tree import ExtraTreeClassifier
from sklearn import linear_model
import lightgbm
import catboost
import xgboost
%matplotlib
###########loading data##########
fdata=pd.read_csv('database_filled.csv',encoding="gbk")
raw_data=fdata.loc[:,[
                      
                       'Zn Content (wt. %)',#0
                      'Co Cotent (wt. %)',#1
                      'Fe Cotent (wt. %)',#2
                      'O Content (at. %)',#4
                      'S Content (at. %)',#5
                      'Content of Pyridinic Species (at. %)',#5
                      'Content of Graphitic Species (at. %)',#6
                      'Content of Oxidized Species (at. %)',#7
                      'Raman ID/IG Ratio',#8
                      'BET Surface Area (m2 g-1)',#9
                      'Micropore Ratio',#10
                      'Mesopore Ratio',#11
                      'Catalyst Loading on RDE/RRDE(mg cm-2)',#12
                      'Rotating Speed (rpm)',#13
                      'HW_0.82'#the classification target#14
                        ]]
###########train test splitting##########
raw_param=raw_data.iloc[:,0:14]
raw_power=raw_data.iloc[:,14]
print('ready')

In [ ]:
def gridsearch(model,param,algorithm_name):
    print('start')
    grid = GridSearchCV(model,param_grid=param,cv=5)
    grid.fit(X_train,y_train)
    print('Best Classifier:',grid.best_params_,'Best Score:', grid.best_score_)
    best_model=grid.best_estimator_
    prediction_train=best_model.predict(X_train)
    prediction_test=best_model.predict(X_test)
    final_result=classification_report(y_test,prediction_test,output_dict=True)
    print(classification_report(y_train,prediction_train))
    print(classification_report(y_test,prediction_test))
    ###########generating a figure##########
    print(algorithm_name)
    print(best_model.feature_importances_)
    rank = [index for index, value in sorted(list(enumerate(best_model.feature_importances_)), key=lambda x:x[1],reverse=True)]
    name_list=[   
                       'Zn Content (wt. %)',#0
                      'Co Cotent (wt. %)',#1
                      'Fe Cotent (wt. %)',#2
                      'O Content (at. %)',#4
                      'S Content (at. %)',#5
                      'Content of Pyridinic Species (at. %)',#5
                      'Content of Graphitic Species (at. %)',#6
                      'Content of Oxidized Species (at. %)',#7
                      'Raman ID/IG Ratio',#8
                      'BET Surface Area (m2 g-1)',#9
                      'Micropore Ratio',#10
                      'Mesopore Ratio',#11
                      'Catalyst Loading on RDE/RRDE(mg cm-2)',#12
                      'Rotating Speed (rpm)'#13
    ]
    plt.bar(range(len(best_model.feature_importances_)), best_model.feature_importances_,color='rgb',tick_label=name_list)
    plt.xticks(rotation=90)
    plt.savefig('%s importance classification 0.82.png' %algorithm_name)


In [ ]:
seed=117
##########LGBM gridsearch CV for best hyperparameter##########
model_LightGBMClassifier=lightgbm.LGBMClassifier(random_state=1,verbose=-1)
param_light = {
 'boosting_type':['gbdt','rf'],
 'learning_rate':[0.01,0.02,0.05,0.1,0.2,1],
 'n_estimators':[100,200,400,800],
 'subsample':[0.5,0.6,0.7,0.8,0.9,1],
 'reg_alpha':[0,0.001,0.01],
 'reg_lambda':[0,0.001,0.01]
}
gridsearch(model_LightGBMClassifier,param_light,'LightGBM')

##########XGBoost gridsearch CV for best hyperparameter##########
model_XGBClassifier=xgboost.XGBClassifier(objective ='reg:squarederror',random_state=1,verbose=0)
param_xg = {
 'booster':['gbtree'],
 'learning_rate':[0.01,0.02,0.05,0.1,0.2,1],
 'n_estimators':[100,200,400,800],
 'max_depth':[5,7,9,11],
 'subsample':[0.5,0.6,0.7,0.8,0.9,1],
 'reg_alpha':[0,0.001,0.01],
 'reg_lambda':[0,0.001,0.01]
}
gridsearch(model_XGBClassifier,param_xg,'XGBoost')

##########CatBoost gridsearch CV for best hyperparameter##########
model_CatClassifier=catboost.CatBoostClassifier(random_state=1,verbose=0)
param_cat = {
 'learning_rate':[0.01,0.05,0.1],
 'n_estimators':[200,400],
 'max_depth':[5,8,11],
 'subsample':[0.5,0.6,0.7,0.8,0.9,1]
}
gridsearch(model_CatClassifier,param_cat,'CatBoost')


###########GradientBoost gridsearch CV for best hyperparameter##########
model_GradientBoostingClassifier = ensemble.GradientBoostingClassifier(random_state=1)
###########defining the parameters dictionary##########
param_GB = {
 'learning_rate':[0.01,0.05,0.1],
 'n_estimators':[200,400],
 'criterion':['friedman_mse','mae','mse'],
 'max_features':['auto','sqrt','log2'],
 'loss':['deviance', 'exponential']
}
gridsearch(model_GradientBoostingClassifier,param_GB,'GradientBoost')

###########RandomForest gridsearch CV for best hyperparameter##########
model_RandomForestClassifier = ensemble.RandomForestClassifier(random_state=1)
###########defining the parameters dictionary##########
param_RF = {
     'n_estimators':[100,200,400,800],
     'criterion':['gini','entropy'],
     'max_features':['auto','sqrt','log2']
}
gridsearch(model_RandomForestClassifier,param_RF,'Random Forest')


###########Extra Tree gridsearch CV for best hyperparameter##########
model_ExtraTreeClassifier = ExtraTreeClassifier(random_state=1)
param_ET = {
        'max_depth':[5,7,9,11],
        'criterion' : ['gini','entropy'],
        'splitter' : [ "best",'random'],
        'max_features':['auto','sqrt','log2']
}
gridsearch(model_ExtraTreeClassifier,param_ET,'Extra Tree')


###########Decision Tree gridsearch CV for best hyperparameter##########
model_DecisionTreeClassifier = tree.DecisionTreeClassifier(random_state=1)
param_DT = {
        'max_depth':[5,7,9,11],
        'criterion' : ['gini','entropy'],
        'splitter' : [ "best",'random'],
        'max_features':['auto','sqrt','log2']
}
gridsearch(model_DecisionTreeClassifier,param_DT,'Decision Tree')


###########AdaBoost gridsearch CV for best hyperparameter##########
model_AdaBoostClassifier = ensemble.AdaBoostClassifier(random_state=1)
param_Ada = {
     'n_estimators':[50,100,200,400,800],
     'learning_rate':[0.01,0.02,0.05,0.1,0.2,1]
}
gridsearch(model_AdaBoostClassifier,param_Ada,'AdaBoost')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(raw_param, raw_power, test_size=.15,random_state=seed)    
search(seed)